# Game Data Project

## Preparing the data

In [92]:
# All imports we will need
import numpy as np
import pandas as pd
import plotly.express as px
import math as mt
import scipy as sp
import streamlit as st
import altair as al
import plotly.colors as pc
from scipy.stats import ttest_ind

Imported what i know ill need and will add as needed

In [93]:
# Read CSV file
game_data = pd.read_csv("../games.csv")

In [94]:
# First viewing of data
game_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


In [95]:
# Lowercase all column headers
game_data.columns = game_data.columns.str.lower()
game_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


Changed all column names to lowercase to make them easier to work with.

In [96]:
# Isolating missing name values
name_missing = game_data[game_data['name'].isna()]
name_missing


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,NaN,GEN,1993.0,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14244,NaN,GEN,1993.0,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


Although i will be dropping these as im not 100% sure i am fairly certain after a good amount opf research that entry 659 is 'Mortal Kombat II' as its the only game in 1993 on the GEN that sold 1.78 million copies in us and also had a healthy sale margin in the EU market and entry 14244 is "Battle Mania Daiginjou" as it was the only game on GEN in 1993 to not receive sales in NA or EU as it was not released there and had only 30000 sale sin jp. However again i will be dropping these lines as they are missing far too much information in multiple categories and we cannot retrieve its name i also believe this is why it was left unfilled.

In [97]:
# Dropping rows with name missing
game_data = game_data.dropna(subset=['name'])
game_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16713 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16444 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.5+ MB


i dropped the previously stated rows 

In [98]:
# Filtering missing values in year of release
yor_missing = game_data[game_data['year_of_release'].isna()]
yor_missing.sample(20)


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
5294,Dragon Ball Z: Budokai Tenkaichi 2 (JP sales),Wii,NaN,Action,0.15,0.05,0.14,0.01,NaN,NaN,NaN
10050,Star Trek: Conquest,PS2,NaN,Strategy,0.06,0.04,0.00,0.01,54.0,9.1,E
12217,Rayman Arena,XB,NaN,Racing,0.05,0.01,0.00,0.00,50.0,tbd,E
609,Space Invaders,2600,NaN,Shooter,2.36,0.14,0.00,0.03,NaN,NaN,NaN
5932,Shrek the Third,DS,NaN,Action,0.26,0.01,0.00,0.02,70.0,6.5,E
5642,Dragster,2600,NaN,Racing,0.30,0.02,0.00,0.00,NaN,NaN,NaN
1650,NASCAR Thunder 2003,PS2,NaN,Racing,0.60,0.46,0.00,0.16,84.0,8.7,E
7387,Move Fitness,PS3,NaN,Sports,0.00,0.16,0.00,0.05,NaN,NaN,NaN
12046,Drake of the 99 Dragons,XB,NaN,Shooter,0.05,0.01,0.00,0.00,22.0,1.8,T
13792,NHL Hitz Pro,GC,NaN,Sports,0.03,0.01,0.00,0.00,81.0,7.8,E


I Went through a couple fixes to the missing values in the years_of_release column such as using the mean/median, dropping the columns, i learned KNN (K-Nearest Neighbors) but i ultimately decided to leave them their as they skewed the data too much and the information itself makes no sense to be imputed as the release date is a fact and not simply data. I am quite certain this data is missing due to human error as some of these games release years are listed in the title.

In [99]:
# Filtering missing values in critic score
critic_missing = game_data[game_data['critic_score'].isna()]
critic_missing.sample(20)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
16127,Kono Aozora ni Yakusoku o: Tenohira no Rakuen,PSP,2009.0,Adventure,0.00,0.00,0.01,0.00,NaN,NaN,NaN
13827,Jojo's Fashion Show: Design in a Dash!,DS,2009.0,Simulation,0.04,0.00,0.00,0.00,NaN,NaN,NaN
10295,FoxKids.com Micro Maniacs Racing,PS,2000.0,Racing,0.06,0.04,0.00,0.01,NaN,NaN,NaN
14840,Young Justice: Legacy,Wii,2013.0,Action,0.00,0.03,0.00,0.00,NaN,NaN,T
10296,Hexen: Beyond Heretic,PS,1997.0,Shooter,0.06,0.04,0.00,0.01,NaN,NaN,NaN
1997,Sonic & Sega All-Stars Racing,DS,2010.0,Racing,0.43,0.50,0.00,0.11,NaN,NaN,NaN
11383,Board Game Classics,GBA,2005.0,Misc,0.06,0.02,0.00,0.00,NaN,tbd,E
15349,Shinken de Watashi ni Koi Shinasai! R,PS3,2012.0,Action,0.00,0.00,0.02,0.00,NaN,NaN,NaN
8903,Power Pro Kun Pocket 9,DS,2006.0,Sports,0.00,0.00,0.15,0.00,NaN,NaN,NaN
6546,Choro Q3,PS,1998.0,Racing,0.00,0.00,0.24,0.02,NaN,NaN,NaN


In [100]:
# Viewing user score column
game_data['user_score'].value_counts()

user_score
tbd    2424
7.8     324
8       290
8.2     282
8.3     254
       ... 
1.3       2
1         2
0.9       2
0         1
9.7       1
Name: count, Length: 96, dtype: int64

Noticed alot fo tbd values need to address that

In [101]:
# Isolating tbd values
tbd_scores = game_data[game_data['user_score'] == 'tbd']
tbd_scores

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
119,Zumba Fitness,Wii,2010.0,Sports,3.45,2.59,0.0,0.66,NaN,tbd,E
301,Namco Museum: 50th Anniversary,PS2,2005.0,Misc,2.08,1.35,0.0,0.54,61.0,tbd,E10+
520,Zumba Fitness 2,Wii,2011.0,Sports,1.51,1.03,0.0,0.27,NaN,tbd,T
645,uDraw Studio,Wii,2010.0,Misc,1.65,0.57,0.0,0.20,71.0,tbd,E
657,Frogger's Adventures: Temple of the Frog,GBA,NaN,Adventure,2.15,0.18,0.0,0.07,73.0,tbd,E
...,...,...,...,...,...,...,...,...,...,...,...
16695,Planet Monsters,GBA,2001.0,Action,0.01,0.00,0.0,0.00,67.0,tbd,E
16697,Bust-A-Move 3000,GC,2003.0,Puzzle,0.01,0.00,0.0,0.00,53.0,tbd,E
16698,Mega Brain Boost,DS,2008.0,Puzzle,0.01,0.00,0.0,0.00,48.0,tbd,E
16704,Plushees,DS,2008.0,Simulation,0.01,0.00,0.0,0.00,NaN,tbd,E


Because i could need these for future calculations i made a separate dataframe for them in case i need them grouped before i drop the tbd for a missing value

In [102]:
# Configuring tbd and changing type to int
game_data['user_score'] = pd.to_numeric(game_data['user_score'], errors='coerce')


game_data['user_score'].value_counts()


user_score
7.8    324
8.0    290
8.2    282
8.3    254
8.5    253
      ... 
1.3      2
1.0      2
0.9      2
0.0      1
9.7      1
Name: count, Length: 95, dtype: int64

I then coerced them into numeric or 'int' which solved the data type problem of them being an object and the tbd to missing value problem

In [103]:
# Creating score dataframe
score_columns = ['critic_score', 'user_score', 'rating']
filtered_scores = game_data[score_columns]
filtered_scores

,critic_score,user_score,rating
0,76.0,8.0,E
1,NaN,NaN,NaN
2,82.0,8.3,E
3,80.0,8.0,E
4,NaN,NaN,NaN
...,...,...,...
16710,NaN,NaN,NaN
16711,NaN,NaN,NaN
16712,NaN,NaN,NaN
16713,NaN,NaN,NaN


In [104]:
# Filtered new dataframe
filtered_scores_cleaned = filtered_scores.dropna(how='all')

filtered_scores_cleaned


,critic_score,user_score,rating
0,76.0,8.0,E
2,82.0,8.3,E
3,80.0,8.0,E
6,89.0,8.5,E
7,58.0,6.6,E
...,...,...,...
16698,48.0,NaN,E
16702,60.0,7.2,E10+
16704,NaN,NaN,E
16705,63.0,5.8,NaN


Then I created a separate dataframe with the critic score user score and rating in one data set because they seemed to be all missing alot of values across the board so i grouped and dropped any row that was missing all 3 and i will use this for the calculations most likely down the line i believe these were left blank as the score do not exist yet indicated by over 2000 entries being labeled 'tbd' i infer there is a certain criteria that nominates a game to be reviewed and the closer you are to that criteria the more swiftly this review is given it would explain why the entries listed as 'tbd' have a more robust well of information in the critic and ratings columns as opposed ot the ones filled with simply 'NaN'

EDIT: i dont think this was a valuable dataframe to make but i will leave it here to show thought process

In [105]:
# Creating a total sales column
sales_columns = [col for col in game_data.columns if 'sales' in col]
game_data['total_sales'] = game_data[sales_columns].sum(axis=1)

game_data.head(20)



,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E,82.54
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN,40.24
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E,35.52
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E,32.77
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN,31.38
5,Tetris,GB,1989.0,Puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN,30.26
6,New Super Mario Bros.,DS,2006.0,Platform,11.28,9.14,6.50,2.88,89.0,8.5,E,29.80
7,Wii Play,Wii,2006.0,Misc,13.96,9.18,2.93,2.84,58.0,6.6,E,28.91
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,14.44,6.94,4.70,2.24,87.0,8.4,E,28.32
9,Duck Hunt,NES,1984.0,Shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN,28.31


I created a total_sales column to for future calculations.

In [120]:
# Yearly release count
yearly_releases = game_data.groupby('year_of_release')['name'].count().reset_index()
yearly_releases.columns = ['year_of_release', 'number_of_games']

yearly_releases = yearly_releases.sort_values(by='year_of_release')

yearly_releases

,year_of_release,number_of_games
0,1980.0,9
1,1981.0,46
2,1982.0,36
3,1983.0,17
4,1984.0,14
5,1985.0,14
6,1986.0,21
7,1987.0,16
8,1988.0,15
9,1989.0,17


## Analyzing the data

In [107]:
# Create a bar chart to visualize the number of games released per year
fig = px.bar(
    yearly_releases,
    x='year_of_release',
    y='number_of_games',
    title='Number of Games Released Per Year',
    labels={'year_of_release': 'Year of Release', 'number_of_games': 'Number of Games'},
)

# Update the layout of the chart
fig.update_layout(
    xaxis=dict(
        tickmode='linear',  # Set tick marks to appear at regular intervals
        tick0=yearly_releases['year_of_release'].min(),
        dtick=1  # Display ticks for every year (interval of 1)
    )
)

# Display the chart
fig.show()



I made a bar graph to illustrate what the distribution of games released per year would look like visually and it shows us that most games released before 1990 were few in number by its post date standards and in this i believe its wise to run any calculations on popularity, revenue, future projections, and other relevant to todays standards calculations be done with games past this date unless clarified that something vintage is our goal. Especially if it relates to the volume of releases. I also believe this serves as a boxplots purpose as the outliers are noticeable (1980-1990).
EDIT: Upon further calculation it seems 1995 is the new latest cutoff

In [108]:
# Sorting the data for the stacked bar chart by total sales
sales_data_long_sorted = (
    game_data.groupby('platform')[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum()
    .assign(total_sales=lambda df: df.sum(axis=1))
    .sort_values(by='total_sales', ascending=False)
    .drop(columns='total_sales')
    .reset_index()
    .melt(id_vars='platform', var_name='region', value_name='sales')
)

# Stacked bar chart with regions as the legend and sorted by total sales
fig_stacked = px.bar(
    sales_data_long_sorted,
    x='platform',
    y='sales',
    color='region',
    title="Sales by Platform Across Regions (Sorted by Total Sales)",
    labels={'sales': 'Sales (in millions)', 'platform': 'Platform'},
    barmode='stack',
    color_discrete_sequence=px.colors.qualitative.Vivid
)
fig_stacked.update_layout(legend_title_text='Region', xaxis_title='Platform', yaxis_title='Sales (in millions)')
fig_stacked.show()

# Total sales bar chart sorted by total sales
total_sales_data_sorted = (
    game_data.groupby('platform')['total_sales'].sum()
    .reset_index()
    .sort_values(by='total_sales', ascending=False)
)

fig_total = px.bar(
    total_sales_data_sorted,
    x='platform',
    y='total_sales',
    title="Total Sales by Platform (Sorted by Total Sales)",
    labels={'total_sales': 'Total Sales (in millions)', 'platform': 'Platform'},
    color_discrete_sequence=["#636EFA"]
)
fig_total.update_layout(legend=None, xaxis_title='Platform', yaxis_title='Total Sales (in millions)')
fig_total.show()


A comparison of sale between platforms the first bar graph is an interactive graph that shows the distribution of sales between regions and platforms and the 2nd is the same however it makes it more clear the max values and how they compare against each other

In [109]:
# Platforms by total sales
top_platforms = (
    game_data.groupby('platform')['total_sales']
    .sum()
    .sort_values(ascending=False)
    .head(20)
    .index
)

top_platforms = game_data[game_data['platform'].isin(top_platforms)]


# Filtering the dataset for years after 1990 and the top 10 platforms
top_platforms = top_platforms[top_platforms['year_of_release'] > 1994]

# Grouping data by year and platform for the distribution
yearly_sales_top_filtered = (
    top_platforms.groupby(['year_of_release', 'platform'])['total_sales']
    .sum()
    .reset_index()
    .sort_values(by=['year_of_release', 'platform'])
)

# Creating the distribution visualization with all years labeled
fig_yearly_distribution_filtered = px.bar(
    yearly_sales_top_filtered,
    x='year_of_release',
    y='total_sales',
    color='platform',
    title="Yearly Sales Distribution for Top Platforms (1995 and Later)",
    labels={'year_of_release': 'Year of Release', 'total_sales': 'Total Sales (in millions)', 'platform': 'Platform'},
    barmode='stack',
    color_discrete_sequence=px.colors.qualitative.Vivid # Added color scheme 
)
fig_yearly_distribution_filtered.update_layout(
    legend_title_text='Platform',
    xaxis=dict(title='Year of Release', tickmode='linear', dtick=1),  # Ensuring all years are labeled
    yaxis_title='Total Sales (in millions)'
)
fig_yearly_distribution_filtered.show()


The top platforms and their sales by year in this bar graph we can see the phases in which the platforms were popular until when or if they start to be overshadowed by a new growing platform. Platforms like "PS", "PS2", "GBA", "DS", "PSP"(briefly), and "Wii" were all a top 3 seller for at least 1 year today have 0 or near 0 sales. We can use these as a good projection of the lifespan of a platform and the time intervals in between new platforms. Platforms generally have a lifespan of 7-10 years, a new platform appears generally every 5-7 years, this tracks as the time it takes for a platform to die after their appearance of another platform is roughly 3-5 years.

If our goal is the project the next couple years of we need to take data from 2006 in our above graph this is also why we have so many platforms in our dataset to better see the newer trends coming in with less values in total simply because they are newer platforms that is the also the maximum average lifespan. if current trends are to be believed then this data was most likely cutoff near the begining of 2016 thus explaining the fewer values despite consistent trends. the newer consoles ("XOne", "PS4", "WiiU") will see a decline in their current sales in the coming 2 year and spend the next 3 years fizzling out and in the next 2-3 years a new console will be introduced i believe these will be profitable as will "XOne" and "PS4" for 2016 and 2017.

In [110]:
# Creating a box plot for global sales broken down by platform
fig_boxplot = px.box(
    game_data,
    x='platform',
    y='total_sales',
    title="Box Plot of Global Sales by Platform",
    labels={'platform': 'Platform', 'total_sales': 'Total Sales (in millions)'},
    color='platform',
    color_discrete_sequence=px.colors.qualitative.Vivid
)
fig_boxplot.update_layout(
    xaxis_title='Platform',
    yaxis_title='Total Sales (in millions)',
    showlegend=False
)
fig_boxplot.show()


there is a very chaotic and inconsistent range of values the sales from platform to platform are relatively significant in their distribution with upper fences ranging from .53 million to 4 million and a varying range of values in between 

In [111]:
average_sales_by_platform = (
    game_data.groupby('platform')['total_sales']
    .mean()
    .reset_index()
    .rename(columns={'total_sales': 'average_sales'})
    .sort_values(by='average_sales', ascending=False)
)

average_sales_by_platform


,platform,average_sales
5,GB,2.606735
11,NES,2.561735
8,GEN,1.050000
23,SNES,0.836987
18,PS4,0.801378
28,X360,0.769746
0,2600,0.729173
17,PS3,0.705973
26,Wii,0.687508
10,N64,0.685517


there is a wide distribution of average sales per platform my prediction is the older consoles have a larger value simply because of lack fo competition in their era and the time in which they have been around.

In [112]:
ps4_data = game_data[game_data['platform'] == 'PS4']

# Checking the relationship between user reviews, professional reviews, and total sales
# Dropping rows with missing values in relevant columns
ps4_reviews_sales = ps4_data.dropna(subset=['user_score', 'critic_score', 'total_sales'])

# Relationship between critic score and sales and trendline
fig_critic_sales = px.scatter(
    ps4_reviews_sales,
    x='critic_score',
    y='total_sales',
    title="Effect of Critic Scores on Sales for PS4 Games",
    labels={'critic_score': 'Critic Score', 'total_sales': 'Total Sales (in millions)'},
    trendline='ols'
)
fig_critic_sales.update_layout(
    xaxis_title='Critic Score',
    yaxis_title='Total Sales (in millions)'
)
fig_critic_sales.show()

# Relationship between user score and sales and trendline
fig_user_sales = px.scatter(
    ps4_reviews_sales,
    x='user_score',
    y='total_sales',
    title="Effect of User Scores on Sales for PS4 Games",
    labels={'user_score': 'User Score', 'total_sales': 'Total Sales (in millions)'},
    trendline='ols'
)
fig_user_sales.update_layout(
    xaxis_title='User Score',
    yaxis_title='Total Sales (in millions)'
)
fig_user_sales.show()


Plotly has a nice trendline parameter that i learned while doing this but i decided to plot a scatter plot and use an OLS trendline to help visualize the impact if any of critic and user scores on sales and it seems that critic scores may not directly impact sales they do however fall in line with sales themselves as sales are typically higher the higher the critic score while the inverse although not as drastic occurs to the user scores as they rise the sales slowly fall. i also observed that the critic scores seem to start under the 0 line for sales in order to create a proper trendline id assume this is from either scores on games that have yet to release therefor have no user score or sale but a critic may very well have access to it or games they rate that low truley do correlate to the sales.

In [113]:
x360_data = game_data[game_data['platform'] == 'X360']

# Checking the relationship between user reviews, professional reviews, and total sales
# Dropping rows with missing values in relevant columns
x360_reviews_sales = x360_data.dropna(subset=['user_score', 'critic_score', 'total_sales'])

# Relationship between critic score and sales for X360 with trendline
fig_critic_sales_x360 = px.scatter(
    x360_reviews_sales,
    x='critic_score',
    y='total_sales',
    title="Effect of Critic Scores on Sales for X360 Games",
    labels={'critic_score': 'Critic Score', 'total_sales': 'Total Sales (in millions)'},
    trendline='ols'
)
fig_critic_sales_x360.update_layout(
    xaxis_title='Critic Score',
    yaxis_title='Total Sales (in millions)'
)
fig_critic_sales_x360.show()

# Relationship between user score and sales for X360 with trendline
fig_user_sales_x360 = px.scatter(
    x360_reviews_sales,
    x='user_score',
    y='total_sales',
    title="Effect of User Scores on Sales for X360 Games",
    labels={'user_score': 'User Score', 'total_sales': 'Total Sales (in millions)'},
    trendline='ols'
)
fig_user_sales_x360.update_layout(
    xaxis_title='User Score',
    yaxis_title='Total Sales (in millions)'
)
fig_user_sales_x360.show()


This is the same exploration on the same games but on the X360 platform and the critic score is strikingly similar and this makes since as id assume they have a certain standard and criteria that qualifies a game as worthy of a high score and it is most likely more uniform than public opinion which brings us to the user scores which sees a different upward trend which is a contrast to what we saw in the "PS4" exploration.

In [114]:
# Analyzing the distribution of games by genre
genre_distribution = game_data['genre'].value_counts().reset_index()
genre_distribution.columns = ['genre', 'count']

# Visualizing the distribution of games by genre

fig_genre_distribution = px.bar(
    genre_distribution,
    x='genre',
    y='count',
    title="Distribution of Games by Genre",
    labels={'genre': 'Genre', 'count': 'Number of Games'},
    color='genre',
    color_discrete_sequence=px.colors.qualitative.Vivid
)
fig_genre_distribution.update_layout(
    xaxis_title='Genre',
    yaxis_title='Number of Games',
    showlegend=False
)
fig_genre_distribution.show()

# Calculating total sales per genre
sales_per_genre = (
    game_data.groupby('genre')['total_sales']
    .sum()
    .reset_index()
    .sort_values(by='total_sales', ascending=False)
)

# Visualizing total sales per genre
fig_sales_genre = px.bar(
    sales_per_genre,
    x='genre',
    y='total_sales',
    title="Total Sales by Genre",
    labels={'genre': 'Genre', 'total_sales': 'Total Sales (in millions)'},
    color='genre',
    color_discrete_sequence=px.colors.qualitative.Vivid
)
fig_sales_genre.update_layout(
    xaxis_title='Genre',
    yaxis_title='Total Sales (in millions)',
    showlegend=False
)
fig_sales_genre.show()


I created a total games and sales to help with this correlation and from the start we can tell that action is the clear favorite in sales and game count followed by sports taking both 2nd spots and as far as sales our 3rd is shooters all games revolving around fast paced, stimulating, overall active games. We can draw some generalizations off of our distributions in sales as stated before a more active, interactive, fast paced game holds a large portion of the sales margin as for the low sales we have a more "comfy" (or casual, slow paced) games such as strategy, puzzle, simulation, adventure.

## Creating a user profile for each region

In [115]:
# Calculating total sales per platform for each region
regional_sales = game_data.groupby('platform')[['na_sales', 'eu_sales', 'jp_sales']].sum()

# Identifying the top five platforms for each region
top_5_na = regional_sales.nlargest(5, 'na_sales')[['na_sales']]
top_5_eu = regional_sales.nlargest(5, 'eu_sales')[['eu_sales']]
top_5_jp = regional_sales.nlargest(5, 'jp_sales')[['jp_sales']]

# Normalizing sales to calculate market share for each region
regional_market_share = regional_sales.div(regional_sales.sum(axis=0), axis=1) * 100

# Extracting market shares for the top platforms
market_share_top_na = regional_market_share.loc[top_5_na.index]
market_share_top_eu = regional_market_share.loc[top_5_eu.index]
market_share_top_jp = regional_market_share.loc[top_5_jp.index]

# Combining data for visualization
market_share_top = pd.concat([
    market_share_top_na.assign(region='NA').reset_index(),
    market_share_top_eu.assign(region='EU').reset_index(),
    market_share_top_jp.assign(region='JP').reset_index()
], ignore_index=True)

# Display the results
market_share_top.sort_values(by=['region', 'platform'])

# Reshaping data for visualization
market_share_melted = market_share_top.melt(
    id_vars=['platform', 'region'],
    value_vars=['na_sales', 'eu_sales', 'jp_sales'],
    var_name='sales_region',
    value_name='market_share'
)

# Filter relevant market shares for top platforms by region
market_share_melted = market_share_melted[market_share_melted['sales_region'] == 'market_share']

# Creating bar plots for each region
fig_na = px.bar(
    market_share_top_na.reset_index(),
    x='platform',
    y='na_sales',
    title="Top Platforms by Market Share in North America",
    labels={'platform': 'Platform', 'na_sales': 'Market Share (%)'},
    color='platform',
    color_discrete_sequence=px.colors.qualitative.Vivid
)
fig_na.update_layout(showlegend=False)

fig_eu = px.bar(
    market_share_top_eu.reset_index(),
    x='platform',
    y='eu_sales',
    title="Top Platforms by Market Share in Europe",
    labels={'platform': 'Platform', 'eu_sales': 'Market Share (%)'},
    color='platform',
    color_discrete_sequence=px.colors.qualitative.Vivid
)
fig_eu.update_layout(showlegend=False)

fig_jp = px.bar(
    market_share_top_jp.reset_index(),
    x='platform',
    y='jp_sales',
    title="Top Platforms by Market Share in Japan",
    labels={'platform': 'Platform', 'jp_sales': 'Market Share (%)'},
    color='platform',
    color_discrete_sequence=px.colors.qualitative.Vivid
)
fig_jp.update_layout(showlegend=False)

# Display plots
fig_na.show()
fig_eu.show()
fig_jp.show()


here we have sectioned and plotted the sales per region and we can see there are a couple differneces form region to region they all do share PS2 in their top 5 but that is the only entry they do all 3 have in common. In na the PS2 and X360 are neck and nexk with Ps2 barely having more in sales. In eu playstation remained a dominate platform followed by X360. Finally in jp we see a whole new platform that the top slot that being DS followed by another new contender the "PS" platform then PS2. every region varied in popularity and trends.

In [116]:
# Calculating total sales per genre for each region
regional_sales_genre = game_data.groupby('genre')[['na_sales', 'eu_sales', 'jp_sales']].sum()

# Identifying the top five genres for each region
top_5_na_genres = regional_sales_genre.nlargest(5, 'na_sales')[['na_sales']]
top_5_eu_genres = regional_sales_genre.nlargest(5, 'eu_sales')[['eu_sales']]
top_5_jp_genres = regional_sales_genre.nlargest(5, 'jp_sales')[['jp_sales']]

# Normalizing sales to calculate market share for each region
regional_genre_market_share = regional_sales_genre.div(regional_sales_genre.sum(axis=0), axis=1) * 100

# Extracting market shares for the top genres
market_share_top_na_genres = regional_genre_market_share.loc[top_5_na_genres.index]
market_share_top_eu_genres = regional_genre_market_share.loc[top_5_eu_genres.index]
market_share_top_jp_genres = regional_genre_market_share.loc[top_5_jp_genres.index]

# Combining data for visualization
market_share_top_genres = pd.concat([
    market_share_top_na_genres.assign(region='NA').reset_index(),
    market_share_top_eu_genres.assign(region='EU').reset_index(),
    market_share_top_jp_genres.assign(region='JP').reset_index()
], ignore_index=True)

# Creating bar plots for each region
fig_na_genres = px.bar(
    market_share_top_na_genres.reset_index(),
    x='genre',
    y='na_sales',
    title="Top Genres by Market Share in North America",
    labels={'genre': 'Genre', 'na_sales': 'Market Share (%)'},
    color='genre',
    color_discrete_sequence=px.colors.qualitative.Vivid
)
fig_na_genres.update_layout(showlegend=False)

fig_eu_genres = px.bar(
    market_share_top_eu_genres.reset_index(),
    x='genre',
    y='eu_sales',
    title="Top Genres by Market Share in Europe",
    labels={'genre': 'Genre', 'eu_sales': 'Market Share (%)'},
    color='genre',
    color_discrete_sequence=px.colors.qualitative.Vivid
)
fig_eu_genres.update_layout(showlegend=False)

fig_jp_genres = px.bar(
    market_share_top_jp_genres.reset_index(),
    x='genre',
    y='jp_sales',
    title="Top Genres by Market Share in Japan",
    labels={'genre': 'Genre', 'jp_sales': 'Market Share (%)'},
    color='genre',
    color_discrete_sequence=px.colors.qualitative.Vivid
)
fig_jp_genres.update_layout(showlegend=False)

# Display plots
fig_na_genres.show()
fig_eu_genres.show()
fig_jp_genres.show()


Ive doen the same with genres per region and in these we see a much more common trend form region to region action and sports are in all regions top 3 genres. both na and eu are almost strikingly similar 1. action 2. sports 3. shooter the only difference being that eus 4th entry is racing whereas in na the 4th entry is platform followed by misc. as the 5th entry. jp has no shooter entry i believe it is due to cultural differences they also have a dominate new contender as their number 1 in role-playing followed by action, sports, platform, and misc.

In [117]:
# Grouping sales data by ESRB rating for each region
esrb_sales = game_data.groupby('rating')[['na_sales', 'eu_sales', 'jp_sales']].sum()

# Normalizing sales to calculate market share for each region
esrb_market_share = esrb_sales.div(esrb_sales.sum(axis=0), axis=1) * 100

# Reshaping data for visualization
esrb_sales_melted = esrb_market_share.reset_index().melt(
    id_vars='rating',
    var_name='region',
    value_name='market_share'
)

# Creating a bar plot to visualize the effect of ESRB ratings on sales in individual regions
fig_esrb_sales = px.bar(
    esrb_sales_melted,
    x='rating',
    y='market_share',
    color='region',
    title="Effect of ESRB Ratings on Sales in Individual Regions",
    labels={'rating': 'ESRB Rating', 'market_share': 'Market Share (%)', 'region': 'Region'},
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.Vivid
)
fig_esrb_sales.update_layout(
    xaxis_title='ESRB Rating',
    yaxis_title='Market Share (%)'
)
fig_esrb_sales.show()


Ive created a bar graph that shows all 3 regions side by side per ESRB rating E and E10+ seem uniform per region however M in na and eu have a noticably larger sale margin than jp they also seem to have a spike in T sales compared to na and eu i also contribute these to cultural differences. I would say that in more explicit content the rating can affect the sales per region

## Testing Hypotheses

In [118]:
# Filtering data for Xbox One and PC
xbox_one_ratings = game_data[(game_data['platform'] == 'XOne') & (game_data['user_score'].notna())]['user_score']
pc_ratings = game_data[(game_data['platform'] == 'PC') & (game_data['user_score'].notna())]['user_score']

# Alpha
alpha = 0.05


# Performing a two-sample t-test
t_stat, p_value = ttest_ind(xbox_one_ratings, pc_ratings, equal_var=False)

# Displaying the results
if p_value <= alpha:
    conclusion = f"Reject the null hypothesis (H0): The average user ratings for Xbox One and PC are significantly different. (p-value = {p_value:.5f})"
else:
    conclusion = f"Fail to reject the null hypothesis (H0): There is no significant difference in the average user ratings for Xbox One and PC. (p-value = {p_value:.5f})"

# Displaying the result
conclusion


'Reject the null hypothesis (H0): The average user ratings for Xbox One and PC are significantly different. (p-value = 0.00000)'

I set up a 2 tailed test to see if the average xbox one and pc sales were the same where the null htpothesis is that the average sales of xbox one and pc are the same (avgXOne = avgPC) and the alternative being (avgXOne != avgPC) and used .05 for my alpha as its standard and based off of our test (the pvalue) and the number was very low meaning we reject the null hypothesis xbox one and pc averages are significantly different

In [119]:
# Filtering data for Xbox One and PC
action_ratings = game_data[(game_data['genre'] == 'Action') & (game_data['user_score'].notna())]['user_score']
sports_ratings = game_data[(game_data['genre'] == 'Sports') & (game_data['user_score'].notna())]['user_score']

# Same alpha

# Performing a two-sample t-test
t_stat_genres, p_value_genres = ttest_ind(action_ratings, sports_ratings, equal_var=False)

# Displaying the results
if p_value_genres <= alpha:
    conclusion_genres = f"Reject the null hypothesis (H0): The average user ratings for Xbox One and PC are significantly different. (p-value = {p_value_genres:.5f})"
else:
    conclusion_genres = f"Fail to reject the null hypothesis (H0): There is no significant difference in the average user ratings for Xbox One and PC. (p-value = {p_value_genres:.5f})"

# Displaying the result
conclusion_genres




'Fail to reject the null hypothesis (H0): There is no significant difference in the average user ratings for Xbox One and PC. (p-value = 0.11484)'

Ive created a very similar test for the new hypothesis of average user ratings for actin and sports games are different and this time we failed to reject the null hypothesis meaning there are no significant differences.

Ive done an EDA on a game sales per region CSV including preparing and cleaning and analyzing the data provided as well as tested a few hypothesis related to the data. We decided what to do with missing values, formatted the column names, and made sure all data was in the correct data type. we then analyzed many different aspects of the data itself including comparisons standalone data and overall trends the data implies. We checked top performing platforms and genres and tested if the data fit certain hypothesis. We've created a better understanding of the data we were given and can use this to make further conclusions if needed.